In [1]:
import ray
from time import time 
from matplotlib import pyplot as plt
from numpy.linalg import norm
from Quantum_functions import *
import numpy as np
from QuantumLib import equilibration_analyser_p
from numpy import allclose


In [2]:
gb = 1024*1024*1024

from qutip import *
# Start Ray.
ray.init()
#ray.init(memory =14*gb, object_store_memory = 12*gb)

2020-04-03 16:58:01,628	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-03 16:58:01,653	INFO resource_spec.py:212 -- Starting Ray with 8.2 GiB memory available for workers and up to 4.12 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-03 16:58:01,961	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.97',
 'redis_address': '192.168.1.97:48312',
 'object_store_address': '/tmp/ray/session_2020-04-03_16-58-01_627771_19362/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-04-03_16-58-01_627771_19362/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-04-03_16-58-01_627771_19362'}

In [ ]:
n = 4
processors = 4
state1 = tensor([basis(2,0)]*n)

alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation

### Testing Trace vs Energy Compare

In [ ]:
start = time()
xs,ys = energy_trace_comp_2d(H1,1,1)
end = time()

single_t = end-start

plt.scatter(xs,ys,s=1)
plt.show()
print(f"Finished executing completed in {end-start}s")

In [ ]:
start = time()
xs,ys = energy_trace_compare_p(H1,1,1,proc =processors)
end = time()

mulit_t  = end-start
print(len(xs))

plt.scatter(xs,ys,s=1)
plt.show()

print(f"Finished executing completed in {end-start}s")


In [ ]:
print(f"System obtains a speedup of {single_t/mulit_t} using {processors} processors" )

### Testing simulation compare

In [10]:
n=12
state1 = tensor([basis(2,0)]*n)
processors = 4
beta0 = lambda n,m: 0
alpha1 = Heisenberg1dRingGen(-1,1,1,n)

H1 = hamiltonian(alpha1,beta0, n)
H1=H1/H1.norm()

assert(H1.isherm)

In [ ]:
start = time()
equilibration_analyser(H1,state1,1e5,200)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_s = end-start

In [ ]:

start = time()
n=13
state1 = tensor([basis(2,0)]*n)
processors = 4
beta0 = lambda n,m: 0
alpha1 = Heisenberg1dRingGen(-1,1,1,n)

H1 = hamiltonian(alpha1,beta0, n)
H1=H1/H1.norm()

print("COnstruction finished")
print(time()-start)

assert(H1.isherm)
energys,states = H1.eigenstates()

print("Ray starting")
print(time()-start)
equilibration_analyser_p(energys,states,state1,0,1e5,200, "TestNew" ,_proc=4)
end=time()
print(f"Finished executing completed in {end-start}s")
sim_m = end-start

COnstruction finished
219.4573905467987
Ray starting
786.432210445404
Basics
0.3539917469024658


In [24]:
print(f"System obtains a speedup of {sim_s/sim_m} using {processors} processors" )

In [ ]:
tresults =[]
for n in range(5,12):
    state1 = tensor([basis(2,0)]*n)
    processors = 4
    beta0 = lambda i,j: 0
    alpha1 = Heisenberg1dRingGen(-1,1,1,n)
    H1 = hamiltonian(alpha1,beta0, n)
    H1=H1/H1.norm()
    assert(H1.isherm)
    
    energys,states = H1.eigenstates()
    start =time()
    equilibration_analyser_p(energys,states,state1,0,1e5,200, "TestNew" ,_proc=4)
    end=time()
    tresults.append(end-start)
    print(f"DONE {n}")

In [7]:
print(tresults)

[1.3501014709472656, 0.9183206558227539, 1.7819147109985352, 3.5457472801208496, 8.197587251663208, 23.829596996307373, 147.182142496109]


In [9]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(range(5,12),tresults)
plt.savefig("NEW")

### Testing Equilibration density operator !

In [8]:
n = 2
processors = 4
initial_state = tensor([basis(2,0)]*n)
state1 = tensor([basis(2,0)]*n)
alpha1 = Heisenberg1dRingGen(1,1,1,n)
alpha0 = lambda n,m,i,j: 0
beta1 = lambda n,i : [0,0,1][i]
beta0 = lambda n,i :0

self_interaction = hamiltonian(alpha1,beta0,n)
self_interaction = self_interaction/self_interaction.norm()
external_interaction = hamiltonian(alpha0,beta1,n)
external_interaction = external_interaction/external_interaction.norm()


perturbation: Qobj = make_hermitian(rand_unitary(2**n, dims = self_interaction.dims))
perturbation = perturbation/perturbation.norm()
epsilon = 0.3

H1 = epsilon*self_interaction + external_interaction + epsilon**2*perturbation
H2 = external_interaction
H3 = external_interaction + epsilon*perturbation
H4 = external_interaction + epsilon*self_interaction